# Conditional MC of 3/2 Model based on QE Scheme

In [1]:
import numpy as np
import sv32_cmc_qe as sv32
import time
import pyfeng as pf
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# parameters
strike = [100.0, 140.0, 70.0]
forward = 100
delta = [1, 1/2, 1/4, 1/8, 1/16, 1/32]

case = np.zeros([3, 6])
case[0] = [1,   0.5, -0.9, 10, 0.04, np.sqrt(0.04)]
case[1] = [0.9, 0.3, -0.5, 15, 0.04, np.sqrt(0.04)]
case[2] = [1,   1,   -0.3, 5,  0.09, np.sqrt(0.09)]

In [3]:
# compute cmc price
for i in range(3):
    vov, kappa, rho, texp, theta, sigma = case[i]
    start = time.time()

    sv32_cmc_qe = sv32.Sv32CondMcQE(vov=vov, kappa=kappa, rho=rho, theta=theta)
    price_cmc = np.zeros([len(delta), len(strike)])
    for d in range(len(delta)):
        price_cmc[d, :] = sv32_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], path=1e5, seed=123456)

    end = time.time()
    
    print('Case %s:' % i)
    print(price_cmc)
    print('Running time is %.3f seconds.' % (end - start) + '\n')

Case 0:
[[22.95314785 10.44093783 38.98364955]
 [23.2425543  10.67162543 39.26731165]
 [23.20965635 10.64143576 39.21865023]
 [22.93527518 10.4758762  38.87971674]
 [22.9298084  10.47613694 38.88556212]
 [23.12806844 10.56484306 39.16893668]]
Running time is 28.535 seconds.

Case 1:
[[28.38562985 16.58649523 42.55740431]
 [28.42977766 16.61705767 42.61118032]
 [28.30786768 16.53990973 42.44717881]
 [28.30763704 16.53355657 42.45931412]
 [28.2822572  16.51219277 42.43468726]
 [28.37245592 16.57440991 42.54708381]]
Running time is 35.012 seconds.

Case 2:
[[25.20424465 13.66791193 39.93642084]
 [25.17249128 13.64813567 39.8960167 ]
 [25.20448092 13.66501972 39.94017581]
 [25.19447764 13.66509173 39.91560506]
 [25.12647967 13.62566786 39.82080577]
 [25.14555403 13.63602067 39.85066992]]
Running time is 11.932 seconds.



In [4]:
# compute std error of cmc price
n = 50
for i in range(3):
    start = time.time()
    vov, kappa, rho, texp, theta, sigma = case[i]

    sv32_cmc_qe = sv32.Sv32CondMcQE(vov=vov, kappa=kappa, rho=rho, theta=theta)
    price_cmc = np.zeros([len(delta), len(strike), n])
    for j in tqdm(range(n)):
        for d in range(len(delta)):
            price_cmc[d, :, j] = sv32_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], path=1e4)

    end = time.time()
    np.set_printoptions(suppress=True)
    print('Case %s:' % i)
    print(price_cmc.mean(axis=2))
    print(price_cmc.std(axis=2))
    print('Running time is %.3f seconds.' % (end - start) + '\n')

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:10<00:00,  2.62s/it]


Case 0:
[[23.12892427 10.58670466 39.14395515]
 [23.11310067 10.5588313  39.14082375]
 [22.99496852 10.50250553 38.97582447]
 [23.09918332 10.54946661 39.12286974]
 [23.08972002 10.54864687 39.10872181]
 [23.07517455 10.53842368 39.08762334]]
[[0.33855107 0.24207028 0.41669954]
 [0.24999649 0.17160117 0.3220097 ]
 [0.32313401 0.2101181  0.41040022]
 [0.36187256 0.2433672  0.46065112]
 [0.28138667 0.17532427 0.36887464]
 [0.34466577 0.22077289 0.45783749]]
Running time is 131.003 seconds.



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [03:29<00:00,  4.19s/it]


Case 1:
[[28.36840496 16.57126759 42.53851457]
 [28.36387599 16.56595286 42.53693055]
 [28.40573459 16.59248576 42.59469761]
 [28.36902132 16.57319228 42.53800659]
 [28.34646388 16.55968213 42.50720542]
 [28.33985771 16.55330382 42.50215674]]
[[0.20651995 0.13662245 0.26931778]
 [0.19592154 0.12349748 0.2674998 ]
 [0.20835949 0.13966642 0.2689695 ]
 [0.20130525 0.13287093 0.2629008 ]
 [0.19065539 0.12728959 0.24776567]
 [0.18380799 0.12136544 0.24172509]]
Running time is 209.458 seconds.



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:27<00:00,  1.75s/it]


Case 2:
[[25.19264404 13.65891243 39.92373074]
 [25.13194016 13.62833076 39.82943228]
 [25.16774189 13.64855021 39.88225421]
 [25.18717431 13.65681153 39.91361937]
 [25.17116275 13.64964289 39.88777631]
 [25.18937323 13.65911294 39.91569048]]
[[0.08101455 0.04620801 0.11685334]
 [0.09553671 0.05406483 0.13874986]
 [0.09393886 0.05364207 0.13600663]
 [0.09935365 0.05603243 0.1442572 ]
 [0.0744847  0.04278011 0.10808451]
 [0.08799603 0.04743762 0.13192094]]
Running time is 87.687 seconds.

